# Импорты

In [1]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import calendar
from tqdm import tqdm
from loguru import logger
from fake_useragent import UserAgent
from pandas import DataFrame
import threading
from datetime import datetime, timedelta

In [2]:
LOGGING = 'logs.log'
FORMAT = "{time} {level} {message}"

# Подбор тегов

In [8]:
url = 'https://asninfo.ru/news/29-12-2023'
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
tree = BeautifulSoup(response.content, 'html.parser')

In [9]:
news = tree.find_all('div', {'class': 'nl__news_block'})

In [10]:
news

[<div class="nl__news_block">
 <a class="nl__news_block__img" href="/news/111641-eksperty-soglasovali-dizayn-kody-gorodov-leningradskoy-oblasti">
 <img alt="шаблон" data-pagespeed-url-hash="1918860975" height="123" onload="pagespeed.CriticalImages.checkImageForCriticality(this);" src="https://asninfo.ru/images/news/1d374071/8e211fe243b1ce66ee626301.jpg" title="Эксперты согласовали дизайн-коды городов Ленинградской области" width="185"/>
 </a>
 <div class="nl__news_block__content">
 <h2 class="nl__news_block__title">
 <a href="/news/111641-eksperty-soglasovali-dizayn-kody-gorodov-leningradskoy-oblasti">Эксперты согласовали дизайн-коды городов Ленинградской области</a>
 </h2>
 <div class="nl__news_block__data">
 <span class="nl__news_block__date">29.12.2023 — 18:00   </span>
 </div>
 <div class="nl__news_block__text news_block__text">Консультативно-экспертный совет по рассмотрению архитектурно-градостроительного облика населенных пунктов, зданий, сооружений Ленинградской области на финал

In [12]:
for new in news:
    print(new.find('a')['href'])
    print(new.find('h2').text)

/news/111641-eksperty-soglasovali-dizayn-kody-gorodov-leningradskoy-oblasti

Эксперты согласовали дизайн-коды городов Ленинградской области

/news/111640-v-kvartale-6-dachnoye-kirovskogo-rayona-zavershilas-rekonstruktsiya-naruzhnogo-osveshcheniya

В квартале 6 Дачное Кировского района завершилась реконструкция наружного освещения

/news/111639-v-problemnom-zhk-oranzh-zapustili-otopleniye

В проблемном ЖК «Оранж» запустили отопление

/news/111638-na-stroyashcheysya-yuzhnoy-ulitse-v-peterburge-idut-raboty-po-ustroystvu-osnovaniya-proyezzhey-chast

На строящейся Южной улице в Петербурге идут работы по устройству основания проезжей части

/news/111637-studenty-praktikanty-sozdadut-kontseptsii-modernizatsii-muzeya-v-kirishi

Студенты-практиканты создадут концепции модернизации музея в Кириши

/news/111636-detskiy-sad-s-basseynom-sdali-v-tsentralnoy-chasti-kolpino

Детский сад с бассейном сдали в центральной части Колпино 

/news/111635-v-moskve-zavershena-rekonstruktsiya-lechebnogo-korpusa-

In [13]:
# Going inside each news item we found
response_inner = requests.get('https://asninfo.ru/news/111622-na-itogovom-gradostroitelnom-sovete-leningradskoy-oblasti-rassmotreli-proyekt-planirovki-territorii-', headers={'User-Agent': UserAgent().firefox})
tree_inner = BeautifulSoup(response_inner.content, 'html.parser')

In [14]:
text = tree_inner.find('div', {'class': 'news_article'})
text.text

'\n\n\n\nсовещание\n\nНа итоговом Градостроительном совете Ленинградской области рассмотрели проект планировки территории в Янино-1 \nИсточник: online47.ru\n\nНа Градостроительном совете Ленинградской области обсудили проект планировки территории и проекта межевания территории в городском поселке Янино-1 Заневского городского поселения Всеволожского района Ленобласти.\n\nКак известно, проект планировки разработан бюро «Фокс Констракшн». Территория расположена в северной части городского поселка Янино-1.\nТак, предусмотрено строительство 345 тысяч квадратных метров жилья для 10 тысяч человек. Строительство планируется в четыре этапа. Предусмотрено строительство трех детских садов на 630 мест и школы на 950 мест. Выдано разрешение на строительство одного из детских садиков, а также площадок для игр детей дошкольного и младшего школьного возраста.\n«В рамках реализации этого проекта компанией будет построен один многоквартирный дом, состоящий из нескольких корпусов, Кроме как строительств

# Парсинг

In [15]:
def get_page(data) -> DataFrame:
    """A function to retrieve data from a web page."""

    logger.add(LOGGING)
    logger.add(LOGGING, format=FORMAT)

    url = f'https://asninfo.ru/news/{data}'
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    tree = BeautifulSoup(response.content, 'html.parser')

    news = tree.find_all('div', {'class': 'nl__news_block'})

    info = []

    for post in news:
        event = threading.Event()
        
        try:
            topic = "Строительство"
        except Exception as e:
            logger.warning(
                f"Failed to get the news topic. Publication date: {data}. "
                f"Exception: {e}")
            pass

        try:
            link = post.find('a')['href']
            link = 'https://asninfo.ru' + link
        except Exception as e:
            logger.warning(
                f"Failed to get the news link. Publication date: {data}. "
                f"Exception: {e}")
            pass

        response_inner = requests.get(link, headers={'User-Agent': UserAgent().firefox})
        tree_inner = BeautifulSoup(response_inner.content, 'html.parser')

        if "https://asninfo.ru/" in link:
            try:
                content = tree_inner.find('div', {'class': 'news_article'}).text
            except Exception as e:
                logger.warning(
                    f"Failed to get the content."
                    f"Publication date: {data}. Exception: {e}")
                pass

        row = {
            'topic': topic,
            'url': link,
            'content': content
        }

        if row['topic'] != 'topic' and \
            row['url'] != 'url' and \
            row['content'] != 'content':
            info.append(row)


    info = pd.DataFrame(info)
    return info

In [ ]:
infa = get_page('2024/01/02', 1)


In [ ]:
infa

In [ ]:
infa = pd.DataFrame()
lenght = 0

start_date = datetime(2023, 12, 29)
end_date = datetime(2020, 1, 1)

progress_bar = tqdm(total=(start_date - end_date).days + 1)

for current_date in pd.date_range(start_date, end_date, freq='-1D'):
    data = current_date.strftime('%d-%m-%Y')
    
    try:
        page_data = get_page(data, )
        infa = pd.concat([infa, page_data], ignore_index=True)
        lenght += len(infa)
        infa.to_csv('builds.csv', index=False, mode='a', header=False)
        infa = pd.DataFrame()
        
    except Exception as e:
        logger.error(f"Failed to get data for: {data}. Exception: {e}.")
        pass

    progress_bar.update(1)

progress_bar.close()

# Исправление

In [ ]:
df = pd.read_csv('lenta_news.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df = df.drop(df[df['topic'] == 'topic'].index)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail(20)

In [ ]:
df = df.drop_duplicates()
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df.to_csv('clean_data.csv', index=False)